Métodos iterativos desarrollados por el grupo 

In [ ]:
#@title Ecuación del problema

"""
  Ecuacion de Shockley con Efecto Early para el circuito del problema. 
"""

import sympy
import math
from sympy.abc import x

V_T = 0.026 # para este valor se asume una temperatura de 300 K
I_S = 2*10**(-16)
V_A = 5 
n = 1
V_BE = 0.7
V_F=5
V_CE=1
R=47000

f_sinValores = "I_S*(E**(V_BE/(n*V_T))-1)*(1-(V_CE/V_A))+(V_F-V_BE)/R"
f_conValores = ("{}*(E**({}/({}*{}))-1)*(1-({}/{}))-({}-{})/{}".format(I_S,"x",n,V_T,V_CE,V_A,V_F,"x",R))

print(f_sinValores)
print(f_conValores)

I_S*(E**(V_BE/(n*V_T))-1)*(1-(V_CE/V_A))+(V_F-V_BE)/R
2e-16*(E**(x/(1*0.026))-1)*(1-(1/5))-(5-x)/47000


In [31]:
#@title Metodo de la Bisección
import sympy
from sympy.abc import x
import time
import math

"""
Entradas: a y b: dos numeros reales para el intervalo
          funcion_text: La funcion del circuito en string

Salidas: xk: Solucion aproximada 
         error: Error absoluto de la función 
         iter: Iteraciones utlizadas
         tiempoEjec: Tiempo de ejecucion

Condicion de parada: iter = iterMax, error < tol
"""
def metodo_biseccion(a,b,funcion_text):
  start_time = time.time()
  tol = 10**-8
  iterMax = 2500

  # Convertir string a simbolico
  f= sympy.lambdify(x,funcion_text, modules=[{'E' :math.exp(1),'e' :math.exp(1),'log':math.log }, 'numpy'])

  # Salidas
  xk = 0
  error = 0
  iter = 0

  # Verificacion T. de Bolzano
  if f(a)*f(b) <= 0:

    for iter in range(iterMax):
      # Definicion del cero de la funcion
      xk = (a+b)/2

      # Verificacion t. de Bolzano en el intervalo actual
      if f(a)*f(xk) <= 0:
        b = xk
      else:
        a = xk

      error = abs(f(xk))

      if error<tol:
        tiempoEjec = time.time() - start_time
        print("Metodo de la biseccion")
        print("Aproximacion del cero: " + str(xk) + " Error: " + str(error) + " Iteraciones: " + str(iter) + " TiempoEjecucion: " + str(tiempoEjec))
        return xk, error, iter, tiempoEjec
    
    tiempoEjec = time.time() - start_time
    print("Metodo de la biseccion")
    print("Aproximacion del cero: " + str(xk) + " Error: " + str(error) + " Iteraciones: " + str(iter) + " TiempoEjecucion: " + str(tiempoEjec))
    return xk, error, iter, tiempoEjec

  else:
    print("La función en ese intervalo no cumple con el Teorema de Bolzano")


# Prueba
metodo_biseccion(0,10, f_conValores)


Metodo de la biseccion
Aproximacion del cero: 0.7038497924804688 Error: 2.599082448868635e-09 Iteraciones: 17 TiempoEjecucion: 0.0026912689208984375


(0.7038497924804688, 2.599082448868635e-09, 17, 0.0026912689208984375)

In [32]:
#@title Método de la Falsa Posición
import sympy
from sympy.abc import x
import time
import math

"""
Entradas: a y b: dos numeros reales para los valores iniciales de xk
          funcion_text: la funcion del circuito en string

Salida: xk : el cero de la función
        error: acercamiento a la solución real
        iter: cantidad de iteraciones utilizadas
        tiempoEjec: tiempo de ejecución
      
Condicion de parada: iter = iterMax, error < tol, denominador = 0
"""
def falsa_posicion(a,b,funcion_text):
  start_time = time.time()
  tol = 10**-8
  iterMax = 2500

  # Convertir texto a simbólico
  f= sympy.lambdify(x,funcion_text, modules=[{'E' :math.exp(1),'e' :math.exp(1),'log':math.log }, 'numpy'])

  # Salidas
  xk = 0
  error = 0
  iter = 0

  # Verificacion T. de Bolzano
  if f(a)*f(b) <= 0:
    for iter in range(0, iterMax):

      # Restriccion para el denominador
      if f(b) - f(a) != 0:
        xk = b - (b-a)/(f(b)-f(a)) * f(b)
      else:
        print("Denominador fue cero")
        return

      # Verificacion del t. de Bolzano en el intervalo actual
      if f(a)*f(xk) <= 0:
        b = xk
      else:
        a = xk

      error = abs(f(xk))

      if error<tol:
        tiempoEjec = time.time() - start_time
        print("Metodo de la Falsa Posicion")
        print("Aproximacion del cero: " + str(xk) + " Error: " + str(error) + " Iteraciones: " + str(iter) + " TiempoEjecucion: " + str(tiempoEjec))
        return xk,error,iter,tiempoEjec

    print("Metodo de la Falsa Posicion")    
    print("Maxima iteracion alcanzada\n")
    tiempoEjec = time.time() - start_time
    print("Aproximacion del cero: " + str(xk) + " Error: " + str(error) + " Iteraciones: " + str(iter) + " TiempoEjecucion: " + str(tiempoEjec))
    return xk,error,iter,tiempoEjec

  else:
    print("La función en ese intervalo no cumple con el Teorema de Bolzano")

# Prueba
falsa_posicion(0.7 ,10, f_conValores)


Metodo de la Falsa Posicion
Maxima iteracion alcanzada

Aproximacion del cero: 0.6999999999999993 Error: 1.2664407661107656e-05 Iteraciones: 2499 TiempoEjecucion: 0.01419520378112793


(0.6999999999999993, 1.2664407661107656e-05, 2499, 0.01419520378112793)

In [33]:
#@title Kou modificado
###Primer metodo iterativo###

from sympy.abc import x
from sympy import diff
import time
import math


"""
divided_diference

Es una funcion auxiliar recursiva para 
la funcion: modified_Kou_method

Entradas: lista: lista de elementos caracteristicos del metodo Kou
          function: funcion del circuito
          derfunction: derivada de la funcion

Salida: elemento cero de la lista o error.
"""
def divided_diference(lista:list,function,derfunction):

  if len(lista)==2 and lista[0]==lista[1]:
    return derfunction(lista[0])/math.factorial(3)

  if len(lista)==0:
    print("error")
    return

  if len(lista)==1:
    return function(lista[0])

  return (divided_diference(lista[1:],function,derfunction)-divided_diference(lista[0:-1],function,derfunction))/(lista[-1]-lista[0])


"""
modified_Kou_method

Entradas: x_0: valor inicial para el metodo
          funcion_text: funcion del circuito en string
          alpha: el valor del angulo para realizar las operaciones

Salidas: x_k: el cero aproximado de la funcion
         error: acercamiento a la solución real
         iter: cantidad de iteraciones utilizadas
         tiempoEjec: tiempo de ejecución

Condicion de parada: iter = iterMax, error < tol, denominador = 0
"""
def modified_Kou_method(x_0, funcion_text, alpha=0):
  start_time = time.time()
  tol = 10**-8
  iterMax = 2500

  # Conversion de string a simbolico
  f = sympy.lambdify(x,funcion_text, modules=[{'E' :math.exp(1),'e' :math.exp(1),'log':math.log }, 'numpy'])
  # Se aplica la derivada al texto
  df_sym = diff(funcion_text,x)
  #print(df_sym)
  # Se convierte la derivada a funcion simbolica
  f_der = sympy.lambdify(x,df_sym, modules=[{'E' :math.exp(1),'e' :math.exp(1),'log':math.log }, 'numpy'])

  # Salidas
  xk = x_0;
  error = 0
  iter = 0

  for iter in range(iterMax):
    
    y_n = xk - f(xk)/(f_der(xk))
    z_n = xk - (alpha*(f(xk)+f(y_n)/f_der(xk))) - (1-alpha)*(f(xk)/f_der(xk))*(f(xk)/(f(xk)-f(y_n)))

    xk = z_n - f(z_n)/(divided_diference([z_n,y_n],f,f_der) + divided_diference([z_n,xk,xk],f,f_der)*(z_n-y_n))

    error = abs(f(xk))

    if error<tol:
      tiempoEjec = time.time() - start_time
      print("Metodo de Kou Modificado")
      print("Aproximacion del cero: " + str(xk) + " Error: " + str(error) + " Iteraciones: " + str(iter) + " TiempoEjecucion: " + str(tiempoEjec))
      return xk,error,iter,tiempoEjec

  print("Metodo de Kou Modificado")
  print("Maxima iterracion alcanzada\n")
  
  tiempoEjec = time.time() - start_time
  print("Aproximacion del cero: " + str(xk) + " Error: " + str(error) + " Iteraciones: " + str(iter) + " TiempoEjecucion: " + str(tiempoEjec))
  return xk,error,iter,tiempoEjec

# Prueba
modified_Kou_method(1, f_conValores)

Metodo de Kou Modificado
Aproximacion del cero: 0.7038506245762752 Error: 3.43960610834455e-10 Iteraciones: 6 TiempoEjecucion: 0.008435249328613281


(0.7038506245762752, 3.43960610834455e-10, 6, 0.008435249328613281)

In [ ]:
#@title Método de Ostrowski

import sympy as sp
from sympy.abc import x
from sympy import diff
import time
import math

"""
Entradas: x_0: valor inicial para el metodo
          funcion_text: funcion del circuito en string

Salidas: x_k: el cero aproximado de la funcion
         error: acercamiento a la solución real
         iter: cantidad de iteraciones utilizadas
         tiempoEjec: tiempo de ejecución

Condicion de parada: iter = iterMax, error < tol, denominador = 0
"""
def ostrowski_method(x_0, funcion_text):
  start_time = time.time()
  tol = 10**-8
  iterMax = 2500  

  # Se considera a1 = a2 = a
  a = 1
  # Se considera m1 = m2 = m
  m = 1
  k = 2

  # Conversión de texto a simbolico
  f= sympy.lambdify(x,funcion_text, modules=[{'E' :math.exp(1),'e' :math.exp(1),'log':math.log }, 'numpy'])
  # Se aplica la derivada al texto
  df_sym=diff(funcion_text,x)
  # Se convierte la derivada en función
  f_der=sympy.lambdify(x,df_sym, modules=[{'E' :math.exp(1),'e' :math.exp(1),'log':math.log }, 'numpy'])

  # Salidas
  x_k = x_0;
  error = 0
  iter = 0

  # Se itera el método hasta iterMax ocasiones
  for iter in range(iterMax):

    # Cálculo de y_n
    # se valida que la derivada en el punto x_k no sea cero porque indefiniría el denominador
    if (f_der(x_k)) != 0:
      y_n = x_k - f(x_k)/f_der(x_k)
    else:
      y_n = x_k
    # Cálculo de H1
    # se verifica que no se indefina la fracción

    if (f(x_k)-2*f(y_n)) != 0:
      H1 = f(y_n) / (f(x_k)-2*f(y_n))
    else:
      H1 = 0
    # Cálculo de z_n
    # se verifica que el denominador no sea cero

    if (f_der(x_k)) != 0:
      z_n = x_k - (1 + H1)*f(x_k)/f_der(x_k)
    else:
      z_n = x_k
    # Cálculo de H2
    # se verifica que el denominador no sea cero
    if (a*f(y_n)+a*f(z_n)) != 0:
      H2 = f(z_n) / (a*f(y_n)+a*f(z_n))
    else:
      H2 = 0
    # Cálculo de x_k
    # se verifica que el denominador no sea cero

    if (f_der(x_k)) != 0:
      x_k = z_n - (m + m*H1 + H2)**k*f(z_n)/f_der(x_k)
    else:
      x_k = z_n

    # se calcula el error absoluto de la función
    error = abs(f(x_k))

    # se condiera la condición de parada con respecto a la tolerancia 
    if error<tol:
      tiempoEjec = time.time() - start_time

      print("Metodo de Ostrowski")
      print("Aproximacion del cero: " + str(x_k) + " Error: " + str(error) + " Iteraciones: " + str(iter) + " TiempoEjecucion: " + str(tiempoEjec))
      return x_k,error,iter,tiempoEjec

  print("Metodo de Ostrowski")
  print("Maxima iteracion alcanzada\n")
  tiempoEjec = time.time() - start_time
  print("Aproximacion del cero: " + str(x_k) + " Error: " + str(error) + " Iteraciones: " + str(iter) + " TiempoEjecucion: " + str(tiempoEjec))
  return x_k,error,iter,tiempoEjec

# Prueba 
ostrowski_method(1,f_conValores)

Metodo de Ostrowski
Aproximacion del cero: 0.7038505235217759 Error: 1.3464455678874332e-11 Iteraciones: 4 TiempoEjecucion: 0.003901958465576172


(0.7038505235217759, 1.3464455678874332e-11, 4, 0.003901958465576172)